In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets

from IPython.display import display
from ipyfilechooser import FileChooser
from time import sleep
import thorlabs_apt as apt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default='notebook' # for vscode ,maybe 'colab' on jupyterlab 

In [ ]:
def apt_device_list():
    print(apt.list_available_devices())

def stage_move_to():
    global stage_serial_num
    global stage_target_pos
    motor = apt.Motor(stage_serial_num)
    motor.move_to(stage_target_pos, blocking=True)

def stage_velocity():
    global stage_serial_num
    global stage_max_velocity
    motor = apt.Motor(stage_serial_num)
    min_v, acc, max_v = motor.get_velocity_parameters()
    motor.set_velocity_parameters(min_v, acc, stage_max_velocity)


    

In [ ]:


def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_apt_device_list = widgets.Button(description='apt_device_list')
    button_stage_move_to = widgets.Button(description='stage_move_to')
    button_stage_velocity = widgets.Button(description='stage_velocity')
    filechooser = FileChooser('./')
    button_load_as_wdata_npy = widgets.Button(description='wdata.npyとして読込')
    button_load_as_tdata_npy = widgets.Button(description='tdata.npyとして読込')
    button_input_field = widgets.Button(description='強制変数反映',disabled=False)

    text_comment = widgets.Text(value='',placeholder='文字を入力',description='comment',disabled=False,style={'description_width': 'initial'})
    text_stage_serial_num = widgets.Text(value='28252436',placeholder='28252436',description='stage_serial_num',disabled=False, style={'description_width': 'initial'})
    float_stage_target_pos = widgets.FloatText(value=0,description='stage_target_pos',disabled=False,style={'description_width': 'initial'})
    float_stage_max_velocity = widgets.FloatText(value=99,description='stage_max_velocity',disabled=False,style={'description_width': 'initial'})



    output = widgets.Output(layour={'border': '1px solid black'})
    def wrapped_func_factory(func,verbose=True):
        def new_func(ui_element):
            with output:
                if verbose: print(f"exec func {func.__name__}")
                func()
                if verbose: print(f"complete {func.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))

    def load_npy_factory(variable_name):
        def load_npy():
            choosed_file_path = filechooser.selected
            global w, t
            if variable_name == 'w':
                w = np.load(choosed_file_path)
                print(f'w.shape={w.shape}')
            elif variable_name == 't':
                t = np.load(choosed_file_path)
                print(f't.shape={t.shape}')
        return load_npy
        
    button_load_as_wdata_npy.on_click(wrapped_func_factory(load_npy_factory('w')))
    button_load_as_tdata_npy.on_click(wrapped_func_factory(load_npy_factory('t')))
    
    def load_input_field():
        global comment
        global stage_serial_num
        global stage_target_pos
        global stage_max_velocity
        comment = text_comment.value
        stage_serial_num = int(text_stage_serial_num.value)
        stage_target_pos = float_stage_target_pos.value
        stage_max_velocity = float_stage_max_velocity.value
    button_input_field.on_click(wrapped_func_factory(load_input_field,verbose=False))
    button_apt_device_list.on_click(wrapped_func_factory(apt_device_list))
    button_stage_move_to.on_click(wrapped_func_factory(stage_move_to))
    button_stage_velocity.on_click(wrapped_func_factory(stage_velocity))
    
    input_fields = [text_comment, text_stage_serial_num, float_stage_target_pos, float_stage_max_velocity]
    for field in input_fields:
        field.continuous_update = False
        field.observe(wrapped_func_factory(load_input_field,verbose=False))

    display(
        widgets.VBox([
            widgets.HBox([button_clear_output]),
            widgets.HBox([button_apt_device_list,button_stage_move_to,button_stage_velocity]),
            widgets.HBox([filechooser, button_load_as_wdata_npy, button_load_as_tdata_npy]),
            widgets.HBox([text_comment, text_stage_serial_num, float_stage_target_pos, button_input_field]),
            widgets.HBox([float_stage_max_velocity]),
            output
        ])
    )


In [ ]:
simple_ui()